# **Extracción, transformación y carga (ETL)**

En la parte superior cargaremos los paquetes

In [175]:
# Cargar paquetes
import pandas as pd
import numpy as np

* Carguemos los datos recopilados para realizar los análisis

In [177]:
# Cargar los datos
datos_recopilados = pd.read_excel('C:/Users/cdeor/OneDrive/Documentos/Python/jb_aet_pm10_25/docs/_data/data_1.xlsx')

# Renombrar las columnas
datos_recopilados.rename(columns={
    'Estacion': 'Estacion',
    'Fecha inicial': 'Fecha',
    'PARÁMETRO': 'Parametro',
    'CONCENTRACIÓN mg/m3': 'Concentración_mg/m3'
}, inplace=True)

datos_recopilados.head(5)

,Estacion,Fecha,Parametro,Concentración_mg/m3
0,LA GLORIA,2022-12-21 00:00:00,PM 10,40.69
1,LA GLORIA,2022-12-20 00:00:00,PM 10,41.01
2,LA GLORIA,2022-12-19 00:00:00,PM 10,40.94
3,LA GLORIA,2022-12-18,PM 10,39.87
4,LA GLORIA,2022-12-17,PM 10,38.79


* Ajustemos la fecha como `datetime`:

In [179]:
# Convertir la columna "Fecha" a formato datetime
datos_recopilados['Fecha'] = pd.to_datetime(datos_recopilados['Fecha'], dayfirst=True)

# Mostrar los datos para verificar
datos_recopilados.head(5)

,Estacion,Fecha,Parametro,Concentración_mg/m3
0,LA GLORIA,2022-12-21,PM 10,40.69
1,LA GLORIA,2022-12-20,PM 10,41.01
2,LA GLORIA,2022-12-19,PM 10,40.94
3,LA GLORIA,2022-12-18,PM 10,39.87
4,LA GLORIA,2022-12-17,PM 10,38.79


* Información de los datos de recopilación

In [180]:
datos_recopilados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Estacion             1352 non-null   object        
 1   Fecha                1352 non-null   datetime64[ns]
 2   Parametro            1352 non-null   object        
 3   Concentración_mg/m3  1352 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 42.4+ KB


* Carguemos los datos de parámetros

In [181]:
# Cargar los datos
datos_parametros = pd.read_excel('C:/Users/cdeor/OneDrive/Documentos/Python/jb_aet_pm10_25/docs/_data/parametros.xlsx')

# Eliminar las columnas especificadas
datos_parametros = datos_parametros.drop(columns=['Unnamed: 1', 'Unnamed: 3'])

# Renombrar las columnas
datos_parametros.rename(columns={
    'FECHA ': 'Fecha',
    'VALOR': 'Valor',
    'UNIDAD': 'Unidad',
    'PARAMETRO': 'Parametros_climaticos'
}, inplace=True)

# Convertir la columna "Fecha" a formato datetime
datos_parametros['Fecha'] = pd.to_datetime(datos_parametros['Fecha'], format='%Y-%m-%d %H:%M', errors='coerce')

datos_parametros.head()

,Fecha,Valor,Unidad,Parametros_climaticos
0,2020-06-17,249.907970,Grados (°),Dirección Viento
1,2020-06-18,106.967656,Grados (°),Dirección Viento
2,2020-06-19,352.443983,Grados (°),Dirección Viento
3,2020-06-20,161.221664,Grados (°),Dirección Viento
4,2020-06-21,192.649780,Grados (°),Dirección Viento


* Información de los datos de parámetros

In [182]:
datos_parametros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  5171 non-null   datetime64[ns]
 1   Valor                  5171 non-null   float64       
 2   Unidad                 5171 non-null   object        
 3   Parametros_climaticos  5171 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 161.7+ KB


* Unamos los datos recopilados y los datos de parámetros

In [183]:
# Unir los DataFrames por la columna "Fecha" usando un merge (right join para mantener todos los datos de df_recopilacion)
df = pd.merge(datos_parametros, datos_recopilados, on='Fecha', how='right')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4951 entries, 0 to 4950
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Fecha                  4951 non-null   datetime64[ns]
 1   Valor                  4951 non-null   float64       
 2   Unidad                 4951 non-null   object        
 3   Parametros_climaticos  4951 non-null   object        
 4   Estacion               4951 non-null   object        
 5   Parametro              4951 non-null   object        
 6   Concentración_mg/m3    4951 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 270.9+ KB


* Veamos si todos los días mediciones de PM 10 y PM 2.5

In [95]:
# Agrupar por fecha y contar las ocurrencias de PM10 y PM2.5
df_grouped = df.groupby('Fecha')['Parametro'].value_counts().unstack().fillna(0)
# df_grouped

# Crear una columna booleana indicando si ambos valores están presentes en fechas consecutivas
df_grouped['Ambos_presentes_consecutivos'] = (df_grouped['PM 10'] > 0) & (df_grouped['PM 2.5'] > 0)

# Filtrar para obtener las fechas donde ambos valores no están presentes en fechas consecutivas
fechas_sin_coincidencia = df_grouped[~df_grouped['Ambos_presentes_consecutivos']].index

# Imprimir los resultados
print("Coinciden:", df_grouped[df_grouped['Ambos_presentes_consecutivos']].index.tolist())
print("No coinciden:", fechas_sin_coincidencia.tolist())

Coinciden: [Timestamp('2020-09-15 00:00:00'), Timestamp('2020-09-16 00:00:00'), Timestamp('2020-09-17 00:00:00'), Timestamp('2020-09-18 00:00:00'), Timestamp('2020-09-19 00:00:00'), Timestamp('2020-09-20 00:00:00'), Timestamp('2020-09-21 00:00:00'), Timestamp('2020-09-22 00:00:00'), Timestamp('2020-09-23 00:00:00'), Timestamp('2020-09-24 00:00:00'), Timestamp('2020-09-25 00:00:00'), Timestamp('2020-09-26 00:00:00'), Timestamp('2020-09-27 00:00:00'), Timestamp('2020-09-28 00:00:00'), Timestamp('2020-09-29 00:00:00'), Timestamp('2020-09-30 00:00:00'), Timestamp('2020-10-01 00:00:00'), Timestamp('2020-10-02 00:00:00'), Timestamp('2020-10-03 00:00:00'), Timestamp('2020-10-04 00:00:00'), Timestamp('2021-09-25 00:00:00'), Timestamp('2021-09-26 00:00:00'), Timestamp('2021-09-27 00:00:00'), Timestamp('2021-09-28 00:00:00'), Timestamp('2021-09-29 00:00:00'), Timestamp('2021-09-30 00:00:00'), Timestamp('2021-10-01 00:00:00'), Timestamp('2021-10-02 00:00:00'), Timestamp('2021-10-03 00:00:00'), Ti

In [82]:
# Contar el número de fechas donde ambos valores no están presentes en fechas consecutivas
num_no_coincidencias = df_grouped[~df_grouped['Ambos_presentes_consecutivos']].shape[0]

print("Número de fechas donde no coinciden:", num_no_coincidencias)

Número de fechas donde no coinciden: 198


* Indexando la fecha para futuras mediciones

In [98]:
df_2 = df.set_index('Fecha')
df_2.head()

,Valor,Unidad,Parametros_climaticos,Estacion,Parametro,Concentración_mg/m3
Fecha,,,,,,
2022-12-21,291.021497,Grados (°),Dirección Viento,LA GLORIA,PM 10,40.69
2022-12-21,98.000000,%,Humedad,LA GLORIA,PM 10,40.69
2022-12-21,32.301221,°C,Temp. Maxima,LA GLORIA,PM 10,40.69
2022-12-21,30.796568,°C,Temp. Mínima,LA GLORIA,PM 10,40.69
2022-12-21,1.852968,m/s,Velocidad Viento,LA GLORIA,PM 10,40.69


## **Datos Faltantes para el PM 10**

* Miraremos los datos faltantes para el PM 10

In [200]:
datos_recopilados.head()

,Estacion,Fecha,Parametro,Concentración_mg/m3
0,LA GLORIA,2022-12-21,PM 10,40.69
1,LA GLORIA,2022-12-20,PM 10,41.01
2,LA GLORIA,2022-12-19,PM 10,40.94
3,LA GLORIA,2022-12-18,PM 10,39.87
4,LA GLORIA,2022-12-17,PM 10,38.79


In [ ]:
df_pm_10 = datos_recopilados[datos_recopilados['Parametro']=='PM 10']
df_pm_10 = df_pm_10.drop_duplicates(subset=['Fecha','Estacion'], keep='first')

df_pm_10.head()

,Estacion,Fecha,Parametro,Concentración_mg/m3
0,LA GLORIA,2022-12-21,PM 10,40.69
1,LA GLORIA,2022-12-20,PM 10,41.01
2,LA GLORIA,2022-12-19,PM 10,40.94
3,LA GLORIA,2022-12-18,PM 10,39.87
4,LA GLORIA,2022-12-17,PM 10,38.79


* Veamos la fecha de inicio y final de la toma de los datos

In [207]:
# Crear un nuevo índice con todas las fechas del mes de septiembre de 2020
fecha_inicio = df_pm_10['Fecha'].min()
fecha_fin = df_pm_10['Fecha'].max()

print(' Fecha de inicio', fecha_inicio)
print(' Fecha de final', fecha_fin)

 Fecha de inicio 2020-09-01 00:00:00
 Fecha de final 2023-12-21 00:00:00


* Ingreso de las fechas faltantes del PM 10

In [208]:
# Crear un índice de fechas continuo
nuevo_indice = pd.date_range(start=fecha_inicio, end=fecha_fin)

# Crear un DataFrame vacío para almacenar los resultados
df_resultado = pd.DataFrame()

# Reindexar por cada estación
for estacion, grupo in df_pm_10.groupby('Estacion'):
    # Reindexar el grupo utilizando el rango de fechas
    grupo_reindexado = (
        grupo.set_index('Fecha')
        .reindex(nuevo_indice)
        .reset_index()
    )
    # Asignar el nombre de la estación al grupo reindexado
    grupo_reindexado['Estacion'] = estacion
    # Rellenar los valores faltantes en "Parametro"
    grupo_reindexado['Parametro'] = grupo_reindexado['Parametro'].fillna('PM 10')
    # Renombrar la columna del índice
    grupo_reindexado.rename(columns={'index': 'Fecha'}, inplace=True)
    # Añadir al DataFrame final
    df_resultado = pd.concat([df_resultado, grupo_reindexado], ignore_index=True)

# Verificar el resultado
df_resultado.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4828 entries, 0 to 4827
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                4828 non-null   datetime64[ns]
 1   Estacion             4828 non-null   object        
 2   Parametro            4828 non-null   object        
 3   Concentración_mg/m3  1012 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 151.0+ KB


In [209]:
df_va = df_resultado.copy()

# Contar la cantidad de datos totales por estación
cantidad_datos_por_estacion = df_va.groupby('Estacion').size().reset_index(name='Cantidad de Datos')

# Contar la cantidad de datos no faltantes en "Concentración_mg/m3" por estación
datos_no_faltantes = df_va.groupby('Estacion')['Concentración_mg/m3'].apply(lambda x: x.notnull().sum()).reset_index(name='Cantidad de datos del PM 10 (no faltantes)')

# Revisar los datos faltantes por estación
missing_values_by_estacion = df_va.groupby('Estacion').apply(lambda x: x.isnull().sum())

# Calcular el porcentaje de valores faltantes por estación
missing_percentage_by_estacion = df_va.groupby('Estacion').apply(lambda x: (x.isnull().mean() * 100).round(2))

# Crear un DataFrame unificado
missing_data_by_estacion = cantidad_datos_por_estacion.copy()
missing_data_by_estacion = missing_data_by_estacion.merge(datos_no_faltantes, on='Estacion', how='left')
missing_data_by_estacion['Valores Faltantes'] = missing_values_by_estacion['Concentración_mg/m3'].values
missing_data_by_estacion['Porcentaje Faltante (%)'] = missing_percentage_by_estacion['Concentración_mg/m3'].values

missing_data_by_estacion

,Estacion,Cantidad de Datos,Cantidad de datos del PM 10 (no faltantes),Valores Faltantes,Porcentaje Faltante (%)
0,LA GLORIA,1207,255,952,78.87
1,TERMINAL,1207,252,955,79.12
2,UNISINU,1207,252,955,79.12
3,UPB,1207,253,954,79.04


## **Datos Faltantes del PM 2.5**

* Miremos los datos faltantes del PM 2.5

In [219]:
df_pm_2_5 = datos_recopilados[datos_recopilados['Parametro']=='PM 2.5']
df_pm_2_5 = df_pm_2_5.drop_duplicates(subset=['Fecha','Estacion'], keep='first')

df_pm_2_5.head()

,Estacion,Fecha,Parametro,Concentración_mg/m3
256,LA GLORIA,2022-11-03,PM 2.5,11.87
257,LA GLORIA,2022-11-02,PM 2.5,13.07
258,LA GLORIA,2022-11-01,PM 2.5,13.84
259,LA GLORIA,2022-10-31,PM 2.5,14.36
260,LA GLORIA,2022-10-30,PM 2.5,12.17


* Ingreso de fechas faltantes del PM 2.5

In [222]:
# Crear un índice de fechas continuo
nuevo_indice = pd.date_range(start=fecha_inicio, end=fecha_fin)

# Crear un DataFrame vacío para almacenar los resultados
df_result = pd.DataFrame()

# Reindexar por cada estación
for estacion, grupo in df_pm_2_5.groupby('Estacion'):
    # Reindexar el grupo utilizando el rango de fechas
    grupo_reindexado = (
        grupo.set_index('Fecha')
        .reindex(nuevo_indice)
        .reset_index()
    )
    # Asignar el nombre de la estación al grupo reindexado
    grupo_reindexado['Estacion'] = estacion
    # Rellenar los valores faltantes en "Parametro"
    grupo_reindexado['Parametro'] = grupo_reindexado['Parametro'].fillna('PM 10')
    # Renombrar la columna del índice
    grupo_reindexado.rename(columns={'index': 'Fecha'}, inplace=True)
    # Añadir al DataFrame final
    df_result = pd.concat([df_result, grupo_reindexado], ignore_index=True)

# Verificar el resultado
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4828 entries, 0 to 4827
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                4828 non-null   datetime64[ns]
 1   Estacion             4828 non-null   object        
 2   Parametro            4828 non-null   object        
 3   Concentración_mg/m3  338 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 151.0+ KB


In [223]:
df_va = df_result.copy()

# Contar la cantidad de datos totales por estación
cantidad_datos_por_estacion = df_va.groupby('Estacion').size().reset_index(name='Cantidad de Datos')

# Contar la cantidad de datos no faltantes en "Concentración_mg/m3" por estación
datos_no_faltantes = df_va.groupby('Estacion')['Concentración_mg/m3'].apply(lambda x: x.notnull().sum()).reset_index(name='Cantidad de datos del PM 2.5 (no faltantes)')

# Revisar los datos faltantes por estación
missing_values_by_estacion = df_va.groupby('Estacion').apply(lambda x: x.isnull().sum())

# Calcular el porcentaje de valores faltantes por estación
missing_percentage_by_estacion = df_va.groupby('Estacion').apply(lambda x: (x.isnull().mean() * 100).round(2))

# Crear un DataFrame unificado
missing_data_by_estacion = cantidad_datos_por_estacion.copy()
missing_data_by_estacion = missing_data_by_estacion.merge(datos_no_faltantes, on='Estacion', how='left')
missing_data_by_estacion['Valores Faltantes'] = missing_values_by_estacion['Concentración_mg/m3'].values
missing_data_by_estacion['Porcentaje Faltante (%)'] = missing_percentage_by_estacion['Concentración_mg/m3'].values

missing_data_by_estacion

,Estacion,Cantidad de Datos,Cantidad de datos del PM 2.5 (no faltantes),Valores Faltantes,Porcentaje Faltante (%)
0,LA GLORIA,1207,78,1129,93.54
1,TERMINAL,1207,93,1114,92.29
2,UNISINU,1207,78,1129,93.54
3,UPB,1207,89,1118,92.63
